In [78]:
# import statements
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [79]:


# importing a sample
arr = pd.read_csv('testWave005_mijn35_R001.csv', header=None, delimiter=";")
# check the shape
input_shape = arr.shape
print(arr.iloc[0])
print(f"The input shape of your array is: {input_shape}")

0                                     Frame
1              Elapsed_Time_In_Milliseconds
2                                      Time
3                                    Hand_X
4                                    Hand_Y
                       ...                 
362       Ring_DIP_Flex_AngularAcceleration
363    Pinky_MCP_Spread_AngularAcceleration
364      Pinky_MCP_Flex_AngularAcceleration
365      Pinky_PIP_Flex_AngularAcceleration
366      Pinky_DIP_Flex_AngularAcceleration
Name: 0, Length: 367, dtype: object
The input shape of your array is: (283, 367)


In [80]:
# make x_train, y_train
# Initialize an empty list to store filenames
x = []
y = [0,0,0,0,1]

# Assuming your integer labels are in 'y_train'
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Specify the base filename (e.g., "test")
base_filename = "testWave005_mijn35_R00"

# Define the total number of files you want (e.g., 50)
num_files = 5

# Generate filenames and add them to the array
for i in range(1, num_files + 1):
    filename = f"{base_filename}{i}.csv"
    file = pd.read_csv(filename, header=0, delimiter=";")
    del file['Time'] # not a float so screws things up
    for col in file.columns:
        file[col] = pd.to_numeric(file[col].replace(",", ""), errors='coerce')
    x.append(file.values)

x = np.array(x) # makes array from list
x = x.astype(np.float32) # regular floats dont work with Tensors
# Print the array of filenames
# print(x)
print(f"The input shape of your array is: {x.shape}")



The input shape of your array is: (5, 282, 366)


In [89]:
model = models.Sequential()
model.add(layers.Conv1D(32, 3, activation='relu', input_shape=(282,366)))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 3, activation='relu'))
model.add(layers.MaxPooling1D(2))
model.add(layers.Conv1D(64, 3, activation='relu'))

In [90]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

In [91]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_36 (Conv1D)          (None, 280, 32)           35168     
                                                                 
 max_pooling1d_23 (MaxPoolin  (None, 140, 32)          0         
 g1D)                                                            
                                                                 
 conv1d_37 (Conv1D)          (None, 138, 64)           6208      
                                                                 
 max_pooling1d_24 (MaxPoolin  (None, 69, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_38 (Conv1D)          (None, 67, 64)            12352     
                                                                 
 flatten_11 (Flatten)        (None, 4288)            

In [92]:
model.compile(optimizer='adam',  # we used adam during machine learning course
              loss='categorical_crossentropy',  # Multiclass classification loss
              metrics=['accuracy'])  # Track accuracy during training

model.fit(x,y,epochs=50)

Epoch 1/50
1/1 [==============================] - 1s 760ms/step - loss: nan - accuracy: 0.8000
Epoch 2/50
1/1 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.8000
Epoch 3/50
1/1 [==============================] - 0s 35ms/step - loss: nan - accuracy: 0.8000
Epoch 4/50
1/1 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8000
Epoch 5/50
1/1 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8000
Epoch 6/50
1/1 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.8000
Epoch 7/50
1/1 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8000
Epoch 8/50
1/1 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8000
Epoch 9/50
1/1 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8000
Epoch 10/50
1/1 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8000
Epoch 11/50
1/1 [==============================] - 0s 8ms/step - l